# Capstone Project
## **Banking in New Mexico, USA, with MESA VERDE BANK**


## Table of contents
___
* Introduction to the Business Problem
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

## Introduction to the Business Problem
_____________

In this data science project, **Mesa Verde Bank** ("MV") is planning to establish bank branch locations in the state of New Mexico.  MV wants to analyze the current landscape of bank branches in New Mexico.  With this information, they will determine ideal locations to target opening new MV branches.

MV will want to understand which competitor banks operate in New Mexico, the number of branches open to the public in the state, and the location of those branches.  By understanding where their competition is (and is not), MV will position itself to gain market share in this new area.

The executives at MV want to use focused criteria in determining the best locations in which to open new branch locations:

- analyze the entire state of New Mexico
- analyze all existing branches for banks in New Mexico (**excluding ATM locations**)
- cross-reference the bank locations against areas of New Mexico with higher per capita income
- determine not only the quantity of branches in areas, but the quantity of unique bank companies

Ultimately, the executives want to position MV in higher income areas of New Mexico, where the existing bank/branch options may be limited.


___
## Data
____

To solve the requests for MV Bank, data will be harvested from multiple sources online.  The harvested data will then be processed and cleansed as need to drill down on the necessary information and to improve data clarity for the analysis  

Using different urls online, the following data for New Mexico will be analyzed:
- per capita income
- zip code information
- county information
- FIPS county data for New Mexico

With Foursquare API, the location and venue information on all banks currently in New Mexico will be called, categorized, and reviewed.


Using the results of the location data and the results of the New Mexico county data, the data can be presented to MV to assist in a recommendation. 

___
Let's begin by installing the necessary libraries to facilitate the project:
___

In [86]:
import pandas as pd
!pip install lxml
print('Pandas installed')
import requests
import numpy as np
print('Numpy installed')

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed')
print('Libraries imported.')

Pandas installed
Numpy installed
Solving environment: done
^C

CondaError: KeyboardInterrupt

failed

CondaError: KeyboardInterrupt

Folium installed
Libraries imported.


___
The county and zip code information for New Mexico can be found at the below url, and the resulting dataframe will be modified to keep only the needed zip-to-county information
___

In [87]:
zip_url = "https://www.zipcodestogo.com/New%20Mexico/"
temp1 = pd.read_html(zip_url)
print("temp1 created: ", len(temp1) , "objects as type: ", type(temp1))

temp1 created:  4 objects as type:  <class 'list'>


___
zip_url was read in as 4 individual lists.  By reviewing those lists, the 3rd object (index = 2) contains the desired information
___

In [88]:
zip_df = temp1[1]
zip_df.head()

,0,1,2,3
0,Zip Codes for the State of New Mexico,Zip Codes for the State of New Mexico,Zip Codes for the State of New Mexico,Zip Codes for the State of New Mexico
1,Zip Code,City,County,Zip Code Map
2,87001,Algodones,Sandoval,View Map
3,87002,Belen,Valencia,View Map
4,87004,Bernalillo,Sandoval,View Map


___
Let's clean up **zip_df** to keep only the Zip and County details.  
Drop the first 2 rows, which are headers but were pulled into the dataframe by pandas...then reset the index.  
Rename the columns, then drop the unnecessary columns.
___

In [89]:
zip_df.drop([0, 1], inplace = True)
zip_df.reset_index(drop = True, inplace = True)
names = ["Zip", "City", "County","Map"]
zip_df.columns = names
zip_df.drop(["City", "Map"], axis = 1, inplace = True)
zip_df.head()

,Zip,County
0,87001,Sandoval
1,87002,Valencia
2,87004,Sandoval
3,87005,Cibola
4,87006,Valencia


___
The per capita income for New Mexico is available at the county level at the url below.

###### (Note: Data is from the 2010 United States Census Data and the 2006-2010 American Community Survey 5-Year Estimates)
___

In [90]:
inc_url = "https://en.wikipedia.org/wiki/List_of_New_Mexico_locations_by_per_capita_income"
temp2 = pd.read_html(inc_url)
print("temp2 created: ", len(temp2) , "objects as type: ", type(temp2))

temp2 created:  5 objects as type:  <class 'list'>


___
inc_url was read in as 5 individual lists.  By reviewing those lists, the 3rd object (index = 2) contains the desired information

Next we'll create and clean **inc_df** to retain the desired information
___

In [91]:
inc_df = temp2[2]
inc_df.dropna(subset = ['Rank'], inplace = True)
inc_df = inc_df.reset_index(drop = True)
inc_df.drop(["Rank", "Medianfamilyincome", "Medianhouseholdincome", "Number ofhouseholds"], axis = 1, inplace = True)
print(inc_df.shape)
inc_df.head()

(33, 3)


,County,Per capitaincome,Population
0,Los Alamos,"$49,474",17950
1,Santa Fe,"$32,188",144170
2,Bernalillo,"$26,143",662564
3,Sandoval,"$25,979",131561
4,Eddy,"$24,587",53829


___
We've established that there are 33 counties in New Mexico.  
We have all the New Mexico zip codes by county in **zip_df**  
We have the income by county in **inc_df**

Since we want to create a choropleth based on the counties and income, we will also need to harvest the FIPS codes for New Mexico.  This will be used as the geoJSON data once we begin creating maps.  
FIPS codes uniquely identify US states and their counties, and by pulling in polygon graphing information (latitudes and longitudes) by FIPS code, we can create a choropleth by county
___

In [92]:
fips_url = "https://ibistest.health.state.nm.us/resource/NMFIPS.html"
temp3 = pd.read_html(fips_url)
print("temp3 created: ", len(temp3) , "tables as type: ", type(temp3))
fips_df = temp3[6]
fips_df.drop("New Mexico Health Region", axis = 1, inplace = True)
print(fips_df.shape)
fips_df.head()

temp3 created:  7 tables as type:  <class 'list'>
(34, 2)


,County,FIPS Code
0,State of New Mexico,35000
1,BERNALILLO,35001
2,CATRON,35003
3,CHAVES,35005
4,CIBOLA,35006


___
Uh-oh, looks like the "County" values of **fips_df** are all Upper-Case, while the "County" values in **inc_df** are a mix of Upper and Lower.  
Let's create an exact match by adding a "low_county" column to each dataframe, with all Lower Case values
___

In [93]:
inc_df["low_county"] = inc_df["County"].str.lower()
fips_df["low_county"] = fips_df["County"].str.lower()
fips_df.head()

,County,FIPS Code,low_county
0,State of New Mexico,35000,state of new mexico
1,BERNALILLO,35001,bernalillo
2,CATRON,35003,catron
3,CHAVES,35005,chaves
4,CIBOLA,35006,cibola


___
Now create a new dataframe **fips_county_income** to merge the FIPS codes with the income by county, and check that the merge went as expected by reviewing the resulting dataframe
___

In [94]:
fips_county_income = fips_df.merge(inc_df, how = "left", on = "low_county")
print(fips_county_income.shape)
fips_county_income

(34, 6)


,County_x,FIPS Code,low_county,County_y,Per capitaincome,Population
0,State of New Mexico,35000,state of new mexico,NaN,NaN,NaN
1,BERNALILLO,35001,bernalillo,Bernalillo,"$26,143",662564.0
2,CATRON,35003,catron,Catron,"$20,895",3725.0
3,CHAVES,35005,chaves,Chaves,"$18,504",65645.0
4,CIBOLA,35006,cibola,Cibola,"$14,712",27213.0
5,COLFAX,35007,colfax,Colfax,"$21,047",13750.0
6,CURRY,35009,curry,Curry,"$19,925",48376.0
7,DEBACA,35011,debaca,NaN,NaN,NaN
8,DONA ANA,35013,dona ana,NaN,NaN,NaN
9,EDDY,35015,eddy,Eddy,"$24,587",53829.0


___
We have an additional row because **fips_df** included an entry for "State of New Mexico".  
Let's delete that entry and reset the index.  

Also note, DEBACA and DONA ANA counties did not load the "per capita income".  
Investigating the cause, it is because the counties are spelled differently in **inc_df**.  

Let's align the data and re-create **fips_county_income** via a merge and check the dtypes of each column
___

In [95]:
fips_df.iloc[7,0] = "DE BACA"
fips_df.iloc[8,0] = "Doña Ana"
fips_df["low_county"] = fips_df["County"].str.lower()

fips_county_income = fips_df.merge(inc_df, how = "left", on = "low_county")
fips_county_income.drop([0], inplace = True)
fips_county_income.reset_index(drop = True, inplace = True)
print(fips_county_income.dtypes)
print(fips_county_income.shape)
fips_county_income

County_x             object
FIPS Code             int64
low_county           object
County_y             object
Per capitaincome     object
Population          float64
dtype: object
(33, 6)


,County_x,FIPS Code,low_county,County_y,Per capitaincome,Population
0,BERNALILLO,35001,bernalillo,Bernalillo,"$26,143",662564.0
1,CATRON,35003,catron,Catron,"$20,895",3725.0
2,CHAVES,35005,chaves,Chaves,"$18,504",65645.0
3,CIBOLA,35006,cibola,Cibola,"$14,712",27213.0
4,COLFAX,35007,colfax,Colfax,"$21,047",13750.0
5,CURRY,35009,curry,Curry,"$19,925",48376.0
6,DE BACA,35011,de baca,De Baca,"$20,769",2022.0
7,Doña Ana,35013,doña ana,Doña Ana,"$18,315",209233.0
8,EDDY,35015,eddy,Eddy,"$24,587",53829.0
9,GRANT,35017,grant,Grant,"$21,164",29514.0


___
Much better.  But notice that column "Per capitaincome" is of type string/object.  
This is no good, as we will need to use that column as the basis of our choropleth shading...and a string is not acceptable in the choropleth function.  
Let's fix that now by removing the $ in the values and casting the column as type float
___

In [96]:
fips_county_income["Per capitaincome"] = fips_county_income["Per capitaincome"].replace('[\$,]', '', regex=True).astype(float)
fips_county_income.dtypes

County_x             object
FIPS Code             int64
low_county           object
County_y             object
Per capitaincome    float64
Population          float64
dtype: object

___
### FOURSQUARE location data  
Now we are on to gathering the location data for this data science analysis.  
Using Foursquare API, we will search New Mexico for all location matches to the search query "Bank".  

Foursquare allows us to search multiple ways, and in this exercise the search will be by zip code.  Good thing we have an available data set **zip_df** created above to reference in this search.

First define the API search credentials, then create a function "json_venues" to search for location data points
___

In [97]:
CLIENT_ID = '0CUBIUQXFQWAGKXPEADPCBLT2ZRQATMKB51WZTX1GUQDNKAE'
CLIENT_SECRET = 'GVZYF4JRMPU13MRGWPDKU4YIQQZZS4JQSJYYIGZR52VNY5RB'
VERSION = '20180706' #YYYMMDD today
search_query = "Bank"

def json_venues(zc):
    results_box = []

    for n in zc:
        get_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            n,
            VERSION,
            search_query)
    
        results_box.append(requests.get(get_url).json())
    return(results_box)

___
Run the custom function, and normalize the resulting json into an object "nm_banks"

To execute this in the desired format, I will explicitly create "nm_banks" by normalizing the first row (index = 0) initially, then append the rest of the rows  using a for loop (beginning at index = 1)
___

In [98]:
a = json_venues(zip_df["Zip"])
nm_banks = json_normalize(a[0]['response']['venues'])

max = len(a)
i_range = np.arange(1, max)

for i in i_range:
    try:
        b = json_normalize(a[i]['response']['venues'])
    except:
        b = []
    
    nm_banks = nm_banks.append(b, ignore_index = True)

print(nm_banks.shape)
nm_banks.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


(6482, 17)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,55d3ae39498e77c8e0931382,U.S. Bank ATM,"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A...",v-1594082252,False,26 Hagon Road,35.416172,-106.401655,"[{'label': 'display', 'lat': 35.41617156, 'lng...",87001,US,Algodones,NM,United States,"[26 Hagon Road, Algodones, NM 87001, United St...",455293791,NaN
1,4e95c7f3775b163c4daa70ea,U.S. Bank Branch,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1594082252,False,221 State Highway 165 Ste F,35.310146,-106.493601,"[{'label': 'display', 'lat': 35.31014636303396...",87043,US,Placitas,NM,United States,"[221 State Highway 165 Ste F, Placitas, NM 870...",379487963,NaN
2,55d4b6d7498ea36af84d050e,U.S. Bank Branch,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1594082252,False,388 W Highway 550,35.317146,-106.539246,"[{'label': 'display', 'lat': 35.3171463, 'lng'...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",376987937,NaN
3,57d3b3f4498e1c6f24bd798b,U.S. Bank,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1594082252,False,388 W Highway 550,35.317147,-106.539244,"[{'label': 'display', 'lat': 35.3171473, 'lng'...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",NaN,NaN
4,4e454db7b0fb93df26f96c5f,U.S. Bank,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1594082252,False,388 W Highway 550,35.320950,-106.551231,"[{'label': 'display', 'lat': 35.32094955444336...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",NaN,NaN


___
Over 6400 banks in New Mexico?  That seems excessive and incorrect.  There's some data cleansing needed here before moving forward.

For instance, by looking at just the first 5 rows of "nm_banks", it appears we have duplicate addresses for the same bank name.  
  
Start with pulling the venue's category from the "categories" column with a try/except function.  
Then focus on only the necessary columns
___

In [99]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


    
nm_banks['categories'] = nm_banks.apply(get_category_type, axis=1)
nm_banks.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,55d3ae39498e77c8e0931382,U.S. Bank ATM,ATM,v-1594082252,False,26 Hagon Road,35.416172,-106.401655,"[{'label': 'display', 'lat': 35.41617156, 'lng...",87001,US,Algodones,NM,United States,"[26 Hagon Road, Algodones, NM 87001, United St...",455293791,NaN
1,4e95c7f3775b163c4daa70ea,U.S. Bank Branch,Bank,v-1594082252,False,221 State Highway 165 Ste F,35.310146,-106.493601,"[{'label': 'display', 'lat': 35.31014636303396...",87043,US,Placitas,NM,United States,"[221 State Highway 165 Ste F, Placitas, NM 870...",379487963,NaN
2,55d4b6d7498ea36af84d050e,U.S. Bank Branch,Bank,v-1594082252,False,388 W Highway 550,35.317146,-106.539246,"[{'label': 'display', 'lat': 35.3171463, 'lng'...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",376987937,NaN
3,57d3b3f4498e1c6f24bd798b,U.S. Bank,Bank,v-1594082252,False,388 W Highway 550,35.317147,-106.539244,"[{'label': 'display', 'lat': 35.3171473, 'lng'...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",NaN,NaN
4,4e454db7b0fb93df26f96c5f,U.S. Bank,Bank,v-1594082252,False,388 W Highway 550,35.320950,-106.551231,"[{'label': 'display', 'lat': 35.32094955444336...",87004,US,Bernalillo,NM,United States,"[388 W Highway 550, Bernalillo, NM 87004, Unit...",NaN,NaN


In [100]:
nm_banks1 = nm_banks[["name", "categories", "location.lat", "location.lng", "location.address", "location.postalCode", "location.state"]]
col_names = ["Bank Name", "Category", "Lat", "Long", "Address", "Zip Code", "State"]
nm_banks1.columns = col_names

nm_banks1

,Bank Name,Category,Lat,Long,Address,Zip Code,State
0,U.S. Bank ATM,ATM,35.416172,-106.401655,26 Hagon Road,87001,NM
1,U.S. Bank Branch,Bank,35.310146,-106.493601,221 State Highway 165 Ste F,87043,NM
2,U.S. Bank Branch,Bank,35.317146,-106.539246,388 W Highway 550,87004,NM
3,U.S. Bank,Bank,35.317147,-106.539244,388 W Highway 550,87004,NM
4,U.S. Bank,Bank,35.320950,-106.551231,388 W Highway 550,87004,NM
...,...,...,...,...,...,...,...
6477,Community 1st Bank Las Vegas,Bank,35.595306,-105.215080,517 6th St,87701,NM
6478,Bank Of Las Vegas,Building,35.594704,-105.216187,622 Douglas Ave,87701,NM
6479,Wells Fargo Bank,Bank,35.594316,-105.216426,701 Douglas Ave,87701,NM
6480,Community First Bank,Bank,35.595906,-105.218818,NaN,87701,NM


___
Immediately we see the Category column includes values other than Bank.  And since Mesa Verde requested that we exclude ATMs, we can filter **nm_banks1** to rows where Category equals Bank.  
Additionally, let's review how many venues are included from states other than New Mexico.  Many zip codes are near the New Mexico border, so it is likely that out-of-state banks we're found by Foursquare during the search
___

In [101]:
state_check = nm_banks1.groupby("State").count()
state_check

,Bank Name,Category,Lat,Long,Address,Zip Code
State,,,,,,
AZ,3,2,3,3,3,3
CO,45,45,45,45,42,44
Colorado,1,1,1,1,0,0
NM,6105,5909,6105,6105,5189,5587
New Mexico,165,164,165,165,0,28
TX,157,157,157,157,129,146
Texas,5,5,5,5,0,0


___
The above results indicate we need to save all rows where "State" equals "NM" or "New Mexico".  
We also see that there are blank attributes for some rows (as evidenced by the fact that there are different numbers across the columns for the same states).  Those na values will be dropped from the list of venues
___

In [102]:
nm_banks_filtered = nm_banks1.loc[(nm_banks1["Category"] == "Bank") & (nm_banks1["State"] == "NM") | (nm_banks1["State"] == "New Mexico")]
nm_banks_filtered.dropna(inplace = True)
nm_banks_filtered = nm_banks_filtered.reset_index(drop = True)
nm_banks_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Bank Name,Category,Lat,Long,Address,Zip Code,State
0,U.S. Bank Branch,Bank,35.310146,-106.493601,221 State Highway 165 Ste F,87043,NM
1,U.S. Bank Branch,Bank,35.317146,-106.539246,388 W Highway 550,87004,NM
2,U.S. Bank,Bank,35.317147,-106.539244,388 W Highway 550,87004,NM
3,U.S. Bank,Bank,35.320950,-106.551231,388 W Highway 550,87004,NM
4,U.S. Bank,Bank,35.320385,-106.551170,388 W Highway 550,87004,NM
...,...,...,...,...,...,...,...
3737,Wells Fargo Bank,Bank,35.176498,-103.724252,302 S 1st St,88401,NM
3738,Farmers & Stockmens Bank,Bank,36.449501,-103.183647,22 Maple St,88415,NM
3739,Community 1st Bank Las Vegas,Bank,35.595306,-105.215080,517 6th St,87701,NM
3740,Wells Fargo Bank,Bank,35.594316,-105.216426,701 Douglas Ave,87701,NM


___
Now we've cleaned the data somewhat, and reduced the venue results from 6508 to 3742 rows.  
But there still appear to be duplicate venues in the list based on the same address being seen on multiple rows.  Also, a single bank branch could have been recorded as nearby multiple zip codes.  
So we will create a "concat" column which combines the "Bank Name" and "Address" and "Zip Code" values.  Then duplicates will be removed 
___

In [103]:
nm_banks_filtered["concat"] = nm_banks_filtered["Bank Name"] + nm_banks_filtered["Address"] + nm_banks_filtered["Zip Code"]
nm_banks_filtered = nm_banks_filtered.drop_duplicates(subset = "concat", ignore_index = True)
nm_banks_filtered

,Bank Name,Category,Lat,Long,Address,Zip Code,State,concat
0,U.S. Bank Branch,Bank,35.310146,-106.493601,221 State Highway 165 Ste F,87043,NM,U.S. Bank Branch221 State Highway 165 Ste F87043
1,U.S. Bank Branch,Bank,35.317146,-106.539246,388 W Highway 550,87004,NM,U.S. Bank Branch388 W Highway 55087004
2,U.S. Bank,Bank,35.317147,-106.539244,388 W Highway 550,87004,NM,U.S. Bank388 W Highway 55087004
3,U.S. Bank,Bank,35.307873,-106.491913,221 State Highway 165 Ste F,87043,NM,U.S. Bank221 State Highway 165 Ste F87043
4,Bank of America,Bank,35.203420,-106.648351,1704 State Highway 528 NW,87114,NM,Bank of America1704 State Highway 528 NW87114
...,...,...,...,...,...,...,...,...
311,BBVA Bank,Bank,33.328464,-105.662143,1710 Sudderth Dr,88345,NM,BBVA Bank1710 Sudderth Dr88345
312,Washington Federal Bank,Bank,33.322989,-105.637573,398 Sudderth Dr,88345,NM,Washington Federal Bank398 Sudderth Dr88345
313,City Bank,Bank,33.320345,-105.630492,135 El Paso Rd,88345,NM,City Bank135 El Paso Rd88345
314,Wells Fargo Bank,Bank,35.176498,-103.724252,302 S 1st St,88401,NM,Wells Fargo Bank302 S 1st St88401


___
That exercise has reduced the bank count to 316 rows, which seems much more accurate than 3742.  While I still see index rows 1 and 2 appear as duplicates, that seems to be a data integrity issue with the "Bank Name" in Foursquare's database.  It will be left as possible noise, and **nm_banks_filtered** will remain the source for our venue information.

Let's modify **nm_banks_filtered** once more to merge in the corresponding "County" names based on **zip_df**.

Then we can also review the location data of these banks in New Mexico, to help Mesa Verde assess their competition and the overall bank landscape
___

In [104]:
nm_banks_filtered = nm_banks_filtered.merge(zip_df, how = "left", left_on = "Zip Code", right_on = "Zip")
nm_banks_filtered.drop("Zip", axis = 1, inplace = True)
nm_banks_filtered

,Bank Name,Category,Lat,Long,Address,Zip Code,State,concat,County
0,U.S. Bank Branch,Bank,35.310146,-106.493601,221 State Highway 165 Ste F,87043,NM,U.S. Bank Branch221 State Highway 165 Ste F87043,Sandoval
1,U.S. Bank Branch,Bank,35.317146,-106.539246,388 W Highway 550,87004,NM,U.S. Bank Branch388 W Highway 55087004,Sandoval
2,U.S. Bank,Bank,35.317147,-106.539244,388 W Highway 550,87004,NM,U.S. Bank388 W Highway 55087004,Sandoval
3,U.S. Bank,Bank,35.307873,-106.491913,221 State Highway 165 Ste F,87043,NM,U.S. Bank221 State Highway 165 Ste F87043,Sandoval
4,Bank of America,Bank,35.203420,-106.648351,1704 State Highway 528 NW,87114,NM,Bank of America1704 State Highway 528 NW87114,Bernalillo
...,...,...,...,...,...,...,...,...,...
311,BBVA Bank,Bank,33.328464,-105.662143,1710 Sudderth Dr,88345,NM,BBVA Bank1710 Sudderth Dr88345,Lincoln
312,Washington Federal Bank,Bank,33.322989,-105.637573,398 Sudderth Dr,88345,NM,Washington Federal Bank398 Sudderth Dr88345,Lincoln
313,City Bank,Bank,33.320345,-105.630492,135 El Paso Rd,88345,NM,City Bank135 El Paso Rd88345,Lincoln
314,Wells Fargo Bank,Bank,35.176498,-103.724252,302 S 1st St,88401,NM,Wells Fargo Bank302 S 1st St88401,Quay


In [105]:
competitors_list = nm_banks_filtered.groupby("Bank Name").count().sort_values(by = "Category", ascending = False)
print("THE FREQUENCY OF BANK BRANCHES BY BRAND IN NEW MEXICO:")
competitors_list

THE FREQUENCY OF BANK BRANCHES BY BRAND IN NEW MEXICO:


,Category,Lat,Long,Address,Zip Code,State,concat,County
Bank Name,,,,,,,,
Wells Fargo Bank,35,35,35,35,35,35,35,35
U.S. Bank Branch,33,33,33,33,33,33,33,33
Bank of America,25,25,25,25,25,25,25,24
Washington Federal Bank,25,25,25,25,25,25,25,25
Bank of the West,24,24,24,24,24,24,24,24
...,...,...,...,...,...,...,...,...
Carlsbad National Bank,1,1,1,1,1,1,1,1
National Bank ATM,1,1,1,1,1,1,1,1
Bank of the Southwest,1,1,1,1,1,1,1,1


In [106]:
bank_zip = nm_banks_filtered.groupby("Zip Code").count().sort_values(by = "Category", ascending = False)
print("THE FREQUENCY OF BANK BRANCHES BY ZIP CODE IN NEW MEXICO:")
bank_zip

THE FREQUENCY OF BANK BRANCHES BY ZIP CODE IN NEW MEXICO:


,Bank Name,Category,Lat,Long,Address,State,concat,County
Zip Code,,,,,,,,
87109,18,18,18,18,18,18,18,18
88011,14,14,14,14,14,14,14,14
87110,13,13,13,13,13,13,13,13
87124,12,12,12,12,12,12,12,12
88201,11,11,11,11,11,11,11,11
...,...,...,...,...,...,...,...,...
87558,1,1,1,1,1,1,1,1
87710,1,1,1,1,1,1,1,1
87901,1,1,1,1,1,1,1,1


___
### Mapping the data
Create a map using folium centered on New Mexico.  
Using the income information, modify with a choropleth.  
Using the venues information, modify with markers representing the 312 bank branches.

The below url contains FIPS data for **all** US counties.  It takes a while to read in and is cumbersome to use.  But since I know that _"35"_ is state FIPS code for New Mexico, I can filter that info into a separate dictionary object to use as the geo data, which will make the data easier to utilize in subsequent coding
___

In [29]:
geo_url = "https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_500k.json"
geo = requests.get(geo_url).json()

___
With the massive data loaded to object "geo", I will create a new object "geo_nm" for geographic details for New Mexico county only.  First initialize "geo_nm" as a dictionary in the same manner as the "geo" get request.

Then use a for loop to iterate through the entire range of "geo" features, appending "geo_nm" with New Mexico features (where "STATE" equals _35_).

Next, check the number of unique features in "geo_nm".  The result should be 33, equivalent to the number of New Mexico counties we had confirmed earlier.

Finally, review the *properties* of first appended feature to determine the name of the field to be used to identify the counties in the choropleth map
___

In [107]:
geo_nm = {'type': 'FeatureCollection', 'features': []}

county_range = np.arange(0, (len(geo["features"])))

for i in county_range:
    if geo["features"][i]["properties"]["STATE"] == "35":
        geo_nm["features"].append(geo["features"][i])
    else:
        pass

print("The number of New Mexico counties appended: ", len(geo_nm["features"]))
print("EXAMPLE of county details in geo_nm:")
geo_nm["features"][0]["properties"]

The number of New Mexico counties appended:  33
EXAMPLE of county details in geo_nm:


{'GEO_ID': '0500000US35013',
 'STATE': '35',
 'COUNTY': '013',
 'NAME': 'Doña Ana',
 'LSAD': 'County',
 'CENSUSAREA': 3807.511,
 'style': {'weight': 1,
  'opacity': 0.7,
  'color': 'black',
  'fillOpacity': 0.6,
  'fillColor': '#fc8d59'},
 'highlight': {}}

___
From the properties above, we know to use the "GEO_ID" value to execute the choropleth.  
However, that value is a longer version of the FIPS code in **fips_county_income**.  Therefore we will add a column to **fips_county_income** which will correlate to the long code in **geo_nm**.

Then we can create our choropleth centered on New Mexico, comparing per capita income by county
___

In [108]:
fips_county_income["FIPS Code"] = fips_county_income["FIPS Code"].astype(str)
fips_county_income["FIPS long code"] = "0500000US" + fips_county_income["FIPS Code"]
print(fips_county_income.shape)
fips_county_income

(33, 7)


,County_x,FIPS Code,low_county,County_y,Per capitaincome,Population,FIPS long code
0,BERNALILLO,35001,bernalillo,Bernalillo,26143.0,662564.0,0500000US35001
1,CATRON,35003,catron,Catron,20895.0,3725.0,0500000US35003
2,CHAVES,35005,chaves,Chaves,18504.0,65645.0,0500000US35005
3,CIBOLA,35006,cibola,Cibola,14712.0,27213.0,0500000US35006
4,COLFAX,35007,colfax,Colfax,21047.0,13750.0,0500000US35007
5,CURRY,35009,curry,Curry,19925.0,48376.0,0500000US35009
6,DE BACA,35011,de baca,De Baca,20769.0,2022.0,0500000US35011
7,Doña Ana,35013,doña ana,Doña Ana,18315.0,209233.0,0500000US35013
8,EDDY,35015,eddy,Eddy,24587.0,53829.0,0500000US35015
9,GRANT,35017,grant,Grant,21164.0,29514.0,0500000US35017


In [109]:
nm_map = folium.Map(location=[34.5, -105.8], zoom_start=7)



nm_map.choropleth(geo_data = geo_nm,
                  data = fips_county_income,
                  columns = ["FIPS long code", "Per capitaincome"],
                  key_on = "feature.properties.GEO_ID",
                  fill_color="RdYlGn",
                  fill_opacity = 0.6,
                  line_opacity = 0.7
                  )
nm_map

___
Using the data from Foursquare, let's overlay the map with markers to indicate the 316 bank branches in New Mexico.

Include pop-ups for each marker with the "Bank Name" 
___

In [110]:
for i in range(0,len(nm_banks_filtered)):
    folium.Marker([nm_banks_filtered.iloc[i]['Lat'], nm_banks_filtered.iloc[i]['Long']]).add_to(nm_map)

nm_map

___
Now we see the overlay of bank branch locations in New Mexico, and where it does and does not correlate with county-level per capita income
___

___
## Methodology
For the Mesa Verde project, we will cluster the existing bank branches to look for patterns and geographic areas where Mesa Verde's competition is currently located.  The DBSCAN clustering method is an appropriate method for this exercise since it examines spatial data, finds high-density clusters, and defines outlier venues based on user-defined settings.

The clustering areas found after DBSCAN analysis will indicate "neighborhoods" where banking businesses are viable and are locations to consider opening MV Bank branches.  But by looking at the outliers, we could analyze for any possible neglected areas to suggest for a new bank opening.

Accompanying this clustering of locations will be the county-level per capita income, which may influence whether or not MV wishes to open a bank in that particular area.

In the "DATA" step of this data science project, New Mexico income, county, and zip code data has been collected.  Additionally the Foursquare venue information was organized and cleansed.

As we move to the "ANALYSIS" step, we will utilize the collected data to look for clusters to assist MV in determining target promising locations for opening bank branches.
___

___
## Analysis

Begin with importing the necessary library for DBSCAN analysis.  

Then further filter our original dataset **nm_banks_filtered** to just the latitude and longitude columns, defined as **dbscan_nm_banks**.  Create the Feature Matrix "XX" using the latitude and longitude.

Next, set the radius/epsilon value and the number of neighborhood samples required to define Core and Border points of a DBSCAN analysis.  By setting the radius as _0.25_ (in relation to lat/long degrees), the radius is approximately 15-20 miles.  And by also lowering the neighborhood samples threshold to _5_, the clusters should be fairly tight.

Then assess the cluster labels, set as "db_labels"
___

In [131]:
#DBSCAN

from sklearn.cluster import DBSCAN 
dbscan_nm_banks = nm_banks_filtered.drop(["Bank Name", "Category", "Address", "Zip Code", "State", "concat", "County"], axis = 1)
XX = dbscan_nm_banks.values[:,:]

radius = 0.25
neighborhood_samples = 5
db = DBSCAN(eps = radius, min_samples = neighborhood_samples).fit(XX)
db_labels = db.labels_

___
Create a new column "DB Labels" and set it equal to the values of the db_labels object above.

Group the "DB Labels" column to assess how many clusters were generated
___

In [136]:
#Assign “labels” to each row of df, as new column
dbscan_nm_banks["DB Labels"] = db_labels

#Check centroid values by getting each cluster’s mean value
dbscan_nm_banks.groupby("DB Labels").count()


,Lat,Long,marker_color
DB Labels,,,
-1,46,46,46
0,134,134,134
1,34,34,34
2,7,7,7
3,10,10,10
4,7,7,7
5,29,29,29
6,6,6,6
7,6,6,6


___
We have 12 unique clusters, as well as the "-1" values which represent the outliers. 

Since we want to overlay this information on the choropleth seen before, we can interpret these clusters as different colored Folium Circle Markers.

Let's create a "marker_color" column with values equal to the "DB Labels".  Then use a for loop with if, elif, and else to set unique colors based on the clusters/label numbers.

The highest concentrations are the 0, 1, and 5 clusters.  Those will appear on the map as red, blue, and white, respectively. 
___

In [134]:
dbscan_nm_banks["marker_color"] = dbscan_nm_banks["DB Labels"]

for i in range(0, len(dbscan_nm_banks)):
    if dbscan_nm_banks["marker_color"][i] == 0:
        dbscan_nm_banks["marker_color"][i] = "red"
    elif dbscan_nm_banks["marker_color"][i] == 1:
        dbscan_nm_banks["marker_color"][i] = "blue"
    elif dbscan_nm_banks["marker_color"][i] == 2:
        dbscan_nm_banks["marker_color"][i] = "green"
    elif dbscan_nm_banks["marker_color"][i] == 3:
        dbscan_nm_banks["marker_color"][i] = "purple"
    elif dbscan_nm_banks["marker_color"][i] == 4:
        dbscan_nm_banks["marker_color"][i] = "orange"
    elif dbscan_nm_banks["marker_color"][i] == 5:
        dbscan_nm_banks["marker_color"][i] = "white"
    elif dbscan_nm_banks["marker_color"][i] == 6:
        dbscan_nm_banks["marker_color"][i] = "pink"
    elif dbscan_nm_banks["marker_color"][i] == 7:
        dbscan_nm_banks["marker_color"][i] = "lightblue"
    elif dbscan_nm_banks["marker_color"][i] == 8:
        dbscan_nm_banks["marker_color"][i] = "lightgreen"
    elif dbscan_nm_banks["marker_color"][i] == 9:
        dbscan_nm_banks["marker_color"][i] = "darkblue"
    elif dbscan_nm_banks["marker_color"][i] == 10:
        dbscan_nm_banks["marker_color"][i] = "gray"
    elif dbscan_nm_banks["marker_color"][i] == 11:
        dbscan_nm_banks["marker_color"][i] = "lightred"
    else:
        dbscan_nm_banks["marker_color"][i] = "black"


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

___
Now create the visualization of this DBSCAN clustered and labeled bank branch locations and the outliers, over a choropleth of New Mexico counties by per capita income
___

In [139]:
dbscan_nm_map = folium.Map(location=[34.5, -105.8], zoom_start=7)


dbscan_nm_map.choropleth(geo_data = geo_nm,
                  data = fips_county_income,
                  columns = ["FIPS long code", "Per capitaincome"],
                  key_on = "feature.properties.GEO_ID",
                  fill_color="RdYlGn",
                  fill_opacity = 0.4,
                  line_opacity = 0.7,
                  legend_name = "Per Capita Income"
                  )

for i in range(0, len(dbscan_nm_banks)):
    folium.CircleMarker([dbscan_nm_banks["Lat"][i], dbscan_nm_banks["Long"][i]], radius=5, color = dbscan_nm_banks["marker_color"][i]).add_to(dbscan_nm_map)


dbscan_nm_map

___
## Results and Discussion
This analysis demonstrates the relationship between the locations of New Mexico banks, their proximity to each other, and the correlation with different counties based on the per capita income of the county.

New Mexico is a large state in regards to geographic area, but not in regards to population.  Therefore seeing the spread of bank branches at first seems to show many large swaths of land with no branches at all was surprising, but in the context of population density some of these "banking gaps" make sense.  Those gaps make further sense when considering the counties with lower per capita income, where competitors may determine the value does not exist to establish bank branches.

There are 12 unique clusters based on the parameters set for the DBSCAN analysis.  One immediate observation is that the 2 highest concentrations of clusters occur in and around the most affluent parts of New Mexico.  But there are multiple other clusters spread throughout the state and these can all be considered "hubs" which have some viability for Mesa Verde.

Another unsurprising realization from the cluster analysis is the proximity of clusters to larger urban areas and major highways.  This again speaks to the likelihood of those areas being able to support banking locations.

The overall analysis and recommendation should push Mesa Verde to **focus on the 12 cluster locations**, specifically the clusters near **Albuquerque and Santa Fe/Los Alamos**, which are located more affluent counties.

But there may be some hidden "outlier" areas where competitors' presence is minimal, but fit the same attributes of existing clusters.  These are locations near larger urban areas and highways, with county-level income levels closer to the median state per capita income.  For instance the city of **Cuba, New Mexico** resides in an affluent county and has only a few branches.  **Carlsbad and Alamogordo** both have little to no bank branch saturation and are in counties near the state's income median.

___
## Conclusion

The purpose of this "Banking in New Mexico" project was to locate areas of New Mexico that Mesa Verde Bank might find amenable based on the current bank branches of competing banks, and also with the context of per capita income in the state.  Analyzing this data through location data, grouping dataframes, and visualizations will aid the stakeholders in coming to an informed decision on optimal target areas.

By extensively harvesting, cleansing, and transforming relevant data we were able to display the landscape of New Mexico banking locations.  There are obvious viable areas as laid out in the clustering results, but there may be some "hidden gem" outlier locations as discussed above that could be of interest to the Mesa Verde team.


The ultimate decisions to be made on optimal bank branch location must be made by leaders and stakeholders at Mesa Verde Bank. Using the data outlined in this analysis should assist that team in intelligently narrowing potential branch locations.
___